In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from torch.utils.data import DataLoader

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-chinese'
word_embedding_model = models.Transformer(model_name, max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
df = pd.read_csv('./data/info_comb.csv')
df

,idx,describe,item_count
0,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n分辨率: 2.8K,2
1,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n性能总结: 高性能轻薄本性能表现一般,2
2,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n办公性能: 一般,2
3,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n散热分析: 惠普星Book Pro1...,2
4,0,屏幕总结: 惠普星Book Pro 14是一款轻薄本\n升级难度: 升级难度较低,2
...,...,...,...
33148,41,噪音: 一般\n续航: 较好\n品牌: 苹果（Apple）\n型号: 苹果 MacBook ...,4
33149,41,"噪音: 一般\n续航: 较好\n品牌: 苹果（Apple）\n外观: ""苹果 MacBook...",4
33150,41,"噪音: 一般\n续航: 较好\n型号: 苹果 MacBook Pro 16\n外观: ""苹果...",4
33151,41,噪音: 一般\n品牌: 苹果（Apple）\n型号: 苹果 MacBook Pro 16\n...,4


In [4]:
train_sentences = df['describe'].values

In [5]:
# Convert train sentences to sentence pairs
train_data = [InputExample(texts=[s, s]) for s in train_sentences]

# DataLoader to batch your data
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    show_progress_bar=True
)

Iteration:  98%|█████████▊| 2029/2073 [25:41<00:33,  1.33it/s]

In [ ]:
model.save('./data/simcse-model')